<a href="https://colab.research.google.com/github/tade0726/DL-2021S2/blob/main/DL_fundamentals_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1, Deep Learning Fundamentals 2021


## todo

### paper

- Intro
- background section of other approches
- Description
- Experimemntal Analysis
- Code
- Conclusion
- Bonus points
  - Improve Perceptron
  - Experimental comparison with other methods with
good analysis


### code

- Create Perceptron model as base
- Create classic models of binary classification (SVM, logistic regression, tree model)
- Improve the Perceptron model



In [ ]:
# prepare libs

! pip install pytorch-lightning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data download

In [ ]:
import requests
from joblib import Memory
from sklearn.datasets import load_svmlight_file
from os.path import join

mem = Memory("./mycache")

def download_file(url, save_path):
  r = requests.get(url, allow_redirects=True)
  with open(save_path, "wb") as file:
    file.write(r.content)
  return save_path

@mem.cache
def get_data(file_path):
    data = load_svmlight_file(file_path)
    return data[0], data[1]

In [ ]:
# config and meta data

data = {
    "raw": "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/diabetes",
    "scale": "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/diabetes_scale"
}


data_dir = "./drive/MyDrive/Colab_Notebooks/Uni/2021S2/DL/DL-asset-1/data"
model_dir = "./drive/MyDrive/Colab_Notebooks/Uni/2021S2/DL/DL-asset-1/model"

data_file_paths  = {}
download_needed = False

In [ ]:
if download_needed:
  for k, url in data.items():
    r = download_file(url, join(data_dir, f"diabetes_{k}"))
    data_file_paths[k] = r
else:
  data_file_paths = {
      "raw": join(data_dir, "diabetes_raw"),
      "scale": join(data_dir, "diabetes_scale")

  }

In [ ]:
data_store = {}

for k, p in data_file_paths.items():
  X, y = get_data(p)
  data_store[k] = (X, y)

## tools functions

In [ ]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

## Baseline: Perceptron

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
import pytorch_lightning as pl
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
import numpy as np

import os


class SP(pl.LightningModule):
  
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(8, 2),
      nn.ReLU(),
    )
    self.ce = nn.CrossEntropyLoss()
    
  def forward(self, x):
    return self.layers(x)
  
  def training_step(self, batch, batch_idx):
    x, y = batch
    x = x.view(x.size(0), -1)
    y_hat = self.layers(x)
    loss = self.ce(y_hat, y.long())
    self.log("train_loss", loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    x = x.view(x.size(0), -1)
    y_hat = self.layers(x)    
    loss = self.ce(y_hat, y.long())

    preds = torch.argmax(y_hat, dim=1)
    acc = accuracy(preds, y.long())

    if stage:
        self.log(f'{stage}_loss', loss, prog_bar=True)
        self.log(f'{stage}_acc', acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    self.evaluate(batch, 'val')

  def test_step(self, batch, batch_idx):
    self.evaluate(batch, 'test')

  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
    return optimizer


class MLP(SP):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(8, 64),
      nn.ReLU(),
      nn.Linear(64, 64),
      nn.ReLU(),
      nn.Linear(64, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 2),
    )
    self.ce = nn.CrossEntropyLoss()


class MlpWithDropout(MLP):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(8, 64),
      nn.ReLU(),
      nn.Linear(64, 64),
      nn.ReLU(),
      nn.Linear(64, 64),
      nn.Dropout(p=0.2),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.Dropout(p=0.2),
      nn.ReLU(),
      nn.Linear(32, 2),
    )
    self.ce = nn.CrossEntropyLoss()


class CNN(MLP):
  
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 3, 2, padding=1),
      nn.Conv2d(3, 2, 2),
      nn.Flatten(),
      nn.Linear(16, 64),
      nn.ReLU(),
      nn.Linear(64, 64),
      nn.ReLU(),
      nn.Linear(64, 64),
      nn.Dropout(p=0.2),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.Dropout(p=0.2),
      nn.ReLU(),
      nn.Linear(32, 2)
    )
    self.ce = nn.CrossEntropyLoss()

  def training_step(self, batch, batch_idx):
    x, y = batch
    x = x.view(-1, 1, 2, 4)
    y_hat = self.layers(x)
    loss = self.ce(y_hat, y.long())
    self.log("train_loss", loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    x = x.view(-1, 1, 2, 4)
    y_hat = self.layers(x)    
    loss = self.ce(y_hat, y.long())

    preds = torch.argmax(y_hat, dim=1)
    acc = accuracy(preds, y.long())

    if stage:
        self.log(f'{stage}_loss', loss, prog_bar=True)
        self.log(f'{stage}_acc', acc, prog_bar=True)


In [ ]:
# load data and split into train set and test set

def create_dataloader(X, y):

  t_X = torch.Tensor(X)
  t_y = torch.Tensor(y)

  dataset = TensorDataset(t_X, t_y)
  dataloader = DataLoader(dataset)
  return dataloader


n_X = data_store["scale"][0].toarray()
n_y = data_store["scale"][1]

n_y = np.where(n_y == -1, 0, n_y)

n_X_train, n_X_test, n_y_train, n_y_test = train_test_split(n_X, n_y, test_size=0.20, random_state=69)
n_X_val, n_X_test, n_y_val, n_y_test = train_test_split(n_X_test, n_y_test, test_size=0.50, random_state=69)


train_data_loader = create_dataloader(n_X_train, n_y_train)
val_data_loader = create_dataloader(n_X_val, n_y_val)
test_data_loader = create_dataloader(n_X_test, n_y_test)

In [ ]:
# convert to a 2D arrays for con

In [ ]:
# size of the data
print("train val test size:")
print(n_y_train.shape[0], n_y_val.shape[0], n_y_test.shape[0])

train val test size:
614 77 77


In [ ]:
# training

@timer
def nn_training_helper(model_name, model, max_epochs, train_data_loader, val_data_loader, test_data_loader):
  global model_dir

  ## define checking point 
  checking_point_path = join(model_dir, model_name)
  checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    dirpath=checking_point_path,
    filename='diabetes-{epoch:02d}-{val_acc:.3f}',
    mode="max"
  )

  # create train
  pl.seed_everything(42)
  trainer = pl.Trainer(auto_scale_batch_size='power', 
                      auto_lr_find=True,
                      gpus=1, 
                      deterministic=True, 
                      max_epochs=max_epochs, 
                      progress_bar_refresh_rate=20,
                      callbacks=[checkpoint_callback])

  trainer.fit(model, train_data_loader, val_data_loader)
  result = trainer.test(dataloaders=test_data_loader)
  return trainer, result

In [ ]:
sp = SP()
t0, r0 = nn_training_helper("sp-v1", sp, 25, train_data_loader, val_data_loader, test_data_loader)
print(r0)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 18    
1 | ce     | CrossEntropyLoss | 0     
--------------------------------------------
18        Trainable params
0         Non-trainable params
18        Total params
0.000     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.37662336230278015, 'test_loss': 0.6843408346176147}
--------------------------------------------------------------------------------
Elapsed time: 88.1038 seconds
[{'test_loss': 0.6843408346176147, 'test_acc': 0.37662336230278015}]


In [ ]:
mlp = MLP()
t1, r1 = nn_training_helper("mlp-v3", mlp, 25, train_data_loader, val_data_loader, test_data_loader)
print(r1)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ./drive/MyDrive/Colab_Notebooks/Uni/2021S2/DL/DL-asset-1/model/mlp-v3 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 11.0 K
1 | ce     | CrossEntropyLoss | 0     
--------------------------------------------
11.0 K    Trainable params
0         Non-trainable params
11.0 K    Total params
0.044     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7792207598686218, 'test_loss': 0.47320428490638733}
--------------------------------------------------------------------------------
Elapsed time: 112.7382 seconds
[{'test_loss': 0.47320428490638733, 'test_acc': 0.7792207598686218}]


In [ ]:
mlp_dropout = MlpWithDropout()
t2, r2 = nn_training_helper("mlp-dropout-v3", mlp_dropout, 25, train_data_loader, val_data_loader, test_data_loader)
print(r2)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ./drive/MyDrive/Colab_Notebooks/Uni/2021S2/DL/DL-asset-1/model/mlp-dropout-v3 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 11.0 K
1 | ce     | CrossEntropyLoss | 0     
--------------------------------------------
11.0 K    Trainable params
0         Non-trainable params
11.0 K    Total params
0.044     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7662337422370911, 'test_loss': 0.47665485739707947}
--------------------------------------------------------------------------------
Elapsed time: 115.8219 seconds
[{'test_loss': 0.47665485739707947, 'test_acc': 0.7662337422370911}]


In [ ]:
cnn = CNN()
t3, r3 = nn_training_helper("cnn-v1", cnn, 25, train_data_loader, val_data_loader, test_data_loader)
print(r3)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ./drive/MyDrive/Colab_Notebooks/Uni/2021S2/DL/DL-asset-1/model/cnn-v1 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 11.6 K
1 | ce     | CrossEntropyLoss | 0     
--------------------------------------------
11.6 K    Trainable params
0         Non-trainable params
11.6 K    Total params
0.046     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7662337422370911, 'test_loss': 0.478452205657959}
--------------------------------------------------------------------------------
Elapsed time: 130.3682 seconds
[{'test_loss': 0.478452205657959, 'test_acc': 0.7662337422370911}]


## contradition machine learning

- logistic regression
- decision tree
- SVM

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


@timer
def train_and_val(model):
  global n_y_test
  model.fit(n_X_train, n_y_train)
  y_pre = model.predict(n_X_test)
  score = accuracy_score(n_y_test, y_pre)
  return score


names = ["Decision Tree", "Linear SVM", "RBF SVM", "Logistic Regresss"]

classifiers = [
  DecisionTreeClassifier(max_depth=5),
  SVC(kernel="linear", C=0.025),
  SVC(gamma=2, C=1),
  LogisticRegression()
]

scores = {
  x: train_and_val(y) for x, y in zip(names, classifiers)
}

Elapsed time: 0.0031 seconds
Elapsed time: 0.0086 seconds
Elapsed time: 0.0141 seconds
Elapsed time: 0.0136 seconds


In [ ]:
scores["sp"] = r0[0]["test_acc"]
scores["mlp"] = r1[0]["test_acc"]
scores["mlp_dropout"] = r2[0]["test_acc"]
scores["mlp_cnn"] = r3[0]["test_acc"]

In [ ]:
scores

{'Decision Tree': 0.7662337662337663,
 'Linear SVM': 0.6363636363636364,
 'Logistic Regresss': 0.7662337662337663,
 'RBF SVM': 0.7662337662337663,
 'mlp': 0.7792207598686218,
 'mlp_cnn': 0.7662337422370911,
 'mlp_dropout': 0.7662337422370911,
 'sp': 0.37662336230278015}